In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## Community

In [2]:
hfcoccps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfsoccps14.csv", sep=",", index_col='ID_S0').sort_index()
hfchdxps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfshdxps14.csv", sep=",", index_col='ID_S0').sort_index()
hfchfaps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfshfaps14.csv", sep=",", index_col='ID_S0').sort_index()
hfcchips16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfschips14.csv", sep=",", index_col='ID_S0').sort_index()
hfccfdps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfscfdps14.csv", sep=",", index_col='ID_S0').sort_index()
hfccelps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfscelps14.csv", sep=",", index_col='ID_S0').sort_index()


In [3]:
#find missing indexes and add them to the others
msng = list(set(hfcoccps16.index) - set(hfchfaps16.index))
for m in msng:
    hfchfaps16 = hfchfaps16.append(pd.Series(name=m))

hfchfaps16 = hfchfaps16.sort_index()

msng = list(set(hfcoccps16.index) - set(hfchdxps16.index))
for m in msng:
    hfchdxps16 = hfchdxps16.append(pd.Series(name=m))

hfchdxps16 = hfchdxps16.sort_index()

msng = list(set(hfcoccps16.index) - set(hfccelps16.index))
for m in msng:
    hfccelps16 = hfccelps16.append(pd.Series(name=m))

hfccelps16 = hfccelps16.sort_index()

In [4]:
cohort = pd.DataFrame()
cohort['ID'] = hfcoccps16.index
cohort = cohort.set_index('ID')

cohort['Age'] = hfcoccps16['AGE'].tolist()

lst = []
for idx in hfcoccps16.index:
    pt = hfcoccps16.loc[idx]
    if pt['SEX'] == 'M': 
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#weight, convert to all be Kg
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20A1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20A'] * 0.45359237)
    else:
        lst.append(pt['HFAA20A'])
        
cohort['Wt_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20B1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20B'] * 0.45359237)
    else:
        lst.append(pt['HFAA20B'])

cohort['Wt_D'] = lst

cohort['BMI_D'] = hfcoccps16['BMI'].tolist()
cohort['EjF'] = hfcoccps16['LVEF_CUR'].tolist()

#NYHA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA24D'] == 'N': 
        lst.append(0)
    else:
        lst.append(pt['HFAA24D'])
cohort['NYHA'] = lst

cohort['TotalHospDays'] = hfcoccps16['LOS'].tolist()

cohort.head(5)

,Age,Gender,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays
ID,,,,,,,,
S106083,66,2,86.000000,88.500000,36.865164,NaN,0,10
S106611,67,1,80.200000,75.000000,24.417188,30.0,4,5
S106644,76,2,57.200000,57.900000,23.346821,NaN,0,5
S106724,78,2,84.640336,81.646627,29.113507,NaN,0,2
S107351,70,1,59.874193,NaN,20.112497,40.0,0,29


In [5]:
#conditions, Ischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6A'] == 'Y' or pt['HFAA6A'] == 'Y' or pt['HFAA11K'] == 'Y' or  pt['HFAA16J']== 'Y' or  pt['HFAA76']== 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ISCH'] = lst

#Nonischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA16K'] == 'Y' or pt['HFAA26C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#AlchE
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AlchE'] = lst

#Angina
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11A'] == 'Y' or pt['HFAA16J'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#ARRH
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA11B2'] == 'Y' or pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ARRH'] = lst

#CARREST
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11D'] == 'Y' or pt['HFAA74'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CARREST'] = lst

#COPD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA10B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

#Depr
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DEPR'] = lst

#Diab
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA12A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#HEPT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA22D'] == 'Y' or pt['HFAA22C'] == 'Y' or pt['HFAA12C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HEPT'] = lst

#HTN
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11J'] == 'Y' or pt['HFAA11L'] == 'Y' or pt['HFAA29D10'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9E'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['MALIG'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA13A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

#Smok
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9H'] == 'Y' or pt['HFAA9G'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#Stroke TIA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14A'] == 'Y' or pt['HFAA75'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['StrokeTIA'] = lst

#VAHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11M'] == 'Y' or pt['HFAA28L'] == 'Y' or pt['HFAA28I'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

#VF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11O'] == 'Y' or pt['HFAA29D4'] == 'Y' or pt['HFAA29D5'] == 'Y' or pt['HFAA29D6'] == 'Y' or pt['HFAA29D7'] == 'Y' or pt['HFAA29D8'] == 'Y' or pt['HFAA32B4'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y' or pt['HFAA26F'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E1'] == 'Y' or pt['HFAA51'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#HTRANS
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

#ICD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E5'] == 'Y' or pt['HFAA11I'] == 'Y' or pt['HFAA47'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#Pace
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E4'] == 'Y' or pt['HFAA49'] == 'Y' or pt['HFAA50'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E2'] == 'Y' or pt['HFAA52'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst


cohort.head(5)

,Age,Gender,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays,ISCH,NonISCH,...,StrokeTIA,VAHD,VF,VHD,VT,CABG,HTRANS,ICD,PACE,PTCI
ID,,,,,,,,,,,,,,,,,,,,,
S106083,66,2,86.000000,88.500000,36.865164,NaN,0,10,0,0,...,0,1,1,0,1,0,0,1,1,0
S106611,67,1,80.200000,75.000000,24.417188,30.0,4,5,1,0,...,0,1,0,0,0,1,0,0,0,0
S106644,76,2,57.200000,57.900000,23.346821,NaN,0,5,0,0,...,0,1,0,0,0,0,0,0,0,0
S106724,78,2,84.640336,81.646627,29.113507,NaN,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
S107351,70,1,59.874193,NaN,20.112497,40.0,0,29,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Labs
cohort['BUN_D'] = hfchfaps16['HFAA45B'].tolist()
cohort['CRT_B'] = hfchfaps16['HFAA44A1'].tolist()

#CRT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if not np.isnan(pt['HFAA44B1']):
        lst.append(pt['HFAA44B1'])
    else:
        lst.append(pt['HFAA44B'])

cohort['CRT_D'] = lst
cohort['HEC_D'] = hfchfaps16['HFAA38B'].tolist()
cohort['HEM_D'] = hfchfaps16['HFAA37B'].tolist()
cohort['SOD_D'] = hfchfaps16['HFAA43B'].tolist()

#Meds
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_D'] = lst

cohort

,Age,Gender,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays,ISCH,NonISCH,...,HEM_D,SOD_D,ACE_B,ACE_D,BET_B,BET_D,NIT_B,NIT_D,DIUR_B,DIUR_D
ID,,,,,,,,,,,,,,,,,,,,,
S106083,66,2,86.000000,88.500000,36.865164,NaN,0,10,0,0,...,10.4,136.0,0,0,1,1,0,0,1,1
S106611,67,1,80.200000,75.000000,24.417188,30.0,4,5,1,0,...,12.5,136.0,0,0,1,1,0,1,1,1
S106644,76,2,57.200000,57.900000,23.346821,NaN,0,5,0,0,...,13.2,132.0,1,0,1,0,0,0,1,0
S106724,78,2,84.640336,81.646627,29.113507,NaN,0,2,0,0,...,10.4,134.0,1,1,1,1,0,0,1,1
S107351,70,1,59.874193,NaN,20.112497,40.0,0,29,0,0,...,9.4,139.0,0,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S210809,78,1,NaN,NaN,NaN,NaN,NaN,77,0,0,...,NaN,NaN,0,0,0,0,0,0,0,0
S210838,80,2,98.400000,94.300000,35.684886,24.0,NaN,11,0,0,...,12.3,137.0,0,0,1,0,0,0,0,0
S210889,80,2,58.968000,55.792000,20.468112,20.0,NaN,5,0,0,...,11.0,136.0,0,0,0,0,1,1,1,1


In [7]:
cohort['HRTRT'] = hfchfaps16['HFAA18A'].tolist()
cohort['BPSYS_B'] = hfchfaps16['HFAA17A'].tolist()
cohort['BPDIAS_B'] = hfchfaps16['HFAA17B'].tolist()
cohort['BPSYS_D'] = hfchfaps16['HFAA17C'].tolist()
cohort['BPDIAS_D'] = hfchfaps16['HFAA17D'].tolist()

cohort['RAP'] = hfchfaps16['HFAA31B1'].tolist()
cohort['PAS'] = hfchfaps16['HFAA31B2'].tolist()
cohort['PCWP'] = hfchfaps16['HFAA31B3'].tolist()
cohort['CO'] = hfchfaps16['HFAA31B4'].tolist()
cohort['CI'] = hfchfaps16['HFAA31B5'].tolist()


idx = sorted(set(cohort.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['MAP_B'] = base
cohort['MAP_D'] = final

#CPI
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = (t['CI'] * t['MAP_B']) / 451
    f = (t['CI'] * t['MAP_D']) / 451
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['CPI_B'] = base
cohort['CPI_D'] = final
    
#PP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PP_B'] = base
cohort['PP_D'] = final

#PPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PPP_B'] = base
cohort['PPP_D'] = final

#SVR
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = 80 * (t['MAP_B']  - t['RAP']) / t['CO']
    f = 80 * (t['MAP_D']  - t['RAP']) / t['CO']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['SVR_B'] = base
cohort['SVR_D'] = final

#RAT
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]
    b = t['RAP'] / t['PCWP']

    base.append(b)

cohort['RAT'] = base


#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['HRTRT']
    f = t['PP_D'] / t['HRTRT']
    c = f - b

    base.append(b)
    final.append(f)
    
cohort['PPRatio_B'] = base
cohort['PPRatio_D'] = final


#SAPi
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['SAPi_B'] = base
cohort['SAPi_D'] = final

#CPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPDIAS_B'] - t['PCWP']
    f = t['BPDIAS_D'] - t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['CPP_B'] = base
cohort['CPP_D'] = final

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['RAP']
    f = t['PP_D'] / t['RAP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PRAPRat_B'] = base
cohort['PRAPRat_D'] = final


cohort

,Age,Gender,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays,ISCH,NonISCH,...,SVR_D,RAT,PPRatio_B,PPRatio_D,SAPi_B,SAPi_D,CPP_B,CPP_D,PRAPRat_B,PRAPRat_D
ID,,,,,,,,,,,,,,,,,,,,,
S106083,66,2,86.000000,88.500000,36.865164,NaN,0,10,0,0,...,NaN,NaN,0.734694,0.571429,NaN,NaN,NaN,NaN,NaN,NaN
S106611,67,1,80.200000,75.000000,24.417188,30.0,4,5,1,0,...,NaN,NaN,1.445946,1.027027,NaN,NaN,NaN,NaN,NaN,NaN
S106644,76,2,57.200000,57.900000,23.346821,NaN,0,5,0,0,...,NaN,NaN,1.126437,0.724138,NaN,NaN,NaN,NaN,NaN,NaN
S106724,78,2,84.640336,81.646627,29.113507,NaN,0,2,0,0,...,NaN,NaN,0.585106,0.606383,NaN,NaN,NaN,NaN,NaN,NaN
S107351,70,1,59.874193,NaN,20.112497,40.0,0,29,0,0,...,NaN,NaN,0.208696,0.356522,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S210809,78,1,NaN,NaN,NaN,NaN,NaN,77,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S210838,80,2,98.400000,94.300000,35.684886,24.0,NaN,11,0,0,...,NaN,NaN,0.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S210889,80,2,58.968000,55.792000,20.468112,20.0,NaN,5,0,0,...,NaN,NaN,0.765625,0.937500,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Make separate datasets

#HEMO
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
hemoDF

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
S106083,66,2,0,NaN,NaN,NaN,0,0,NaN,NaN,...,72.0,0.507042,0,NaN,NaN,0.734694,0,NaN,NaN,NaN
S106083,66,2,0,NaN,NaN,NaN,0,0,NaN,NaN,...,56.0,0.437500,0,NaN,NaN,0.571429,0,NaN,NaN,NaN
S106611,67,1,0,30.0,NaN,NaN,0,0,NaN,NaN,...,107.0,0.557292,0,NaN,NaN,1.445946,0,NaN,NaN,NaN
S106611,67,1,0,30.0,NaN,NaN,0,0,NaN,NaN,...,76.0,0.513514,0,NaN,NaN,1.027027,0,NaN,NaN,NaN
S106644,76,2,0,NaN,NaN,NaN,0,0,NaN,NaN,...,98.0,0.601227,0,NaN,NaN,1.126437,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S210889,80,2,0,20.0,NaN,NaN,0,0,NaN,NaN,...,60.0,0.444444,0,NaN,NaN,0.937500,0,NaN,NaN,NaN
S210906,82,2,0,NaN,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
S210906,82,2,0,NaN,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN


In [9]:
#Get only patients with hemo
hemoTest = hemoDF[['RAP', 'PAS', 'PCWP', 'CO', 'CI']]
hemoComposite = hemoDF.loc[~(np.isnan(hemoTest)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)

(56, 28)


,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
S113719,55,1,0,10.0,23.0,29.0,0,0,20.0,NaN,...,22.0,0.220000,0,NaN,1.150,0.224490,0,1.100000,58.0,0.956522
S113719,55,1,0,10.0,23.0,29.0,0,0,20.0,NaN,...,49.0,0.392000,0,NaN,1.150,0.500000,0,2.450000,56.0,2.130435
S114046,61,1,0,NaN,14.0,35.0,0,0,16.0,3.8,...,13.0,0.183099,0,2014.035088,0.875,0.097744,0,0.812500,42.0,0.928571
S114046,61,1,0,NaN,14.0,35.0,0,0,16.0,3.8,...,18.0,0.285714,0,1663.157895,0.875,0.135338,0,1.125000,29.0,1.285714
S115069,60,1,0,NaN,3.0,23.0,0,0,3.0,3.3,...,130.0,0.764706,0,4694.949495,1.000,1.262136,0,43.333333,37.0,43.333333


In [10]:
#save to file
hemoComposite.to_csv("../Data Validation/ARIC/Original DataFrames/HemoAricComm.csv")

In [ ]:
#MAke Hemo Labels
labels = pd.DataFrame()
labels['ID'] = hemoComposite.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hemoComposite.index:
    pt = hfchfaps16.loc[idx]
    pt1 = hfccelps16.loc[idx]
    if pt1['CELB06'] == 'Y' or pt1['CELB12'] == 'Y' or pt['HFAA57'] == 'Y' or pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/HemoLabelsAricComm.csv")

In [11]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Death
ID,
S113719,0
S113719,0
S114046,0
S114046,0
S115069,0
S115069,0
S115472,0
S115472,0
S117776,0


In [12]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/HemoLabelsAricComm.csv")

In [13]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
S106083,66,2,0,86.000,0.000000,0,10,0,0,0,...,1,NaN,70.0,142.0,0,0,188.666667,72.0,0.507042,0.734694
S106083,66,2,0,88.500,36.865164,0,10,0,0,0,...,1,NaN,72.0,128.0,0,0,176.000000,56.0,0.437500,0.571429
S106611,67,1,0,80.200,0.000000,0,5,4,0,0,...,1,30.0,85.0,192.0,0,0,248.666667,107.0,0.557292,1.445946
S106611,67,1,0,75.000,24.417188,0,5,4,0,0,...,1,30.0,72.0,148.0,0,0,196.000000,76.0,0.513514,1.027027
S106644,76,2,0,57.200,0.000000,0,5,0,0,1,...,1,NaN,65.0,163.0,0,0,206.333333,98.0,0.601227,1.126437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S210889,80,2,0,55.792,20.468112,0,5,NaN,0,1,...,1,20.0,75.0,135.0,0,0,185.000000,60.0,0.444444,0.937500
S210906,82,2,0,NaN,0.000000,0,3,NaN,0,0,...,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
S210906,82,2,0,NaN,NaN,0,3,NaN,0,0,...,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [14]:
#save to file
allData.to_csv("../Data Validation/ARIC/Original DataFrames/AllDataAricComm.csv")

In [15]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = hfcoccps16.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    pt1 = hfccelps16.loc[idx]
    if pt1['CELB06'] == 'Y' or pt1['CELB12'] == 'Y' or pt['HFAA57'] == 'Y' or pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

,Death
ID,
S106083,0
S106611,0
S106644,1
S106724,0
S107351,0
...,...
S210809,0
S210838,0
S210889,0


In [16]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/LabelsAricComm.csv")